In [1]:
import operator
import math
import numpy as np
import io
from itertools import combinations, permutations, islice, product, count
from functools import reduce
from collections import defaultdict, namedtuple
from operator import methodcaller

In [2]:
def get_input(day, year = 2017, type = 'in'):
    return open("data/{}/{}.{}".format(year, day, type))

<center><h1><a href='https://adventofcode.com/2017/day/1'>Day 1: Inverse Captcha</a></h1></center>

### Part One:

In [3]:
def day_1_1(numbers):
    numbers_size = len(numbers)
    return sum([numbers[i] for i in range(numbers_size) if numbers[i] == numbers[(i+1)%numbers_size]])

### Part Two:

In [4]:
def day_1_2(numbers):
    numbers_size = len(numbers)
    return sum([numbers[i] for i in range(numbers_size) if numbers[i] == numbers[(i+numbers_size//2)%numbers_size]])

### Results:

In [5]:
numbers = list(map(int, list(''.join(get_input(1).readline().split()))))

print('Part One: ', day_1_1(numbers))
print('Part Two: ', day_1_2(numbers))

Part One:  1047
Part Two:  982


<center><h1><a href='https://adventofcode.com/2017/day/2'>Day 2: Corruption Checksum</a></h1></center>

### Part One:

In [6]:
def day_2_1(spreedsheet):
    return sum([max(row) - min(row) for row in spreedsheet])

### Part Two:

In [7]:
def day_2_2(spreedsheet):
    return sum([a//b for row in spreedsheet for a,b in permutations(row, 2) if a%b == 0])

### Results:

In [8]:
spreedsheet = list(map(lambda line: list(map(int, line.split())), get_input(2).readlines()))

print('Part One: ', day_2_1(spreedsheet))
print('Part Two: ', day_2_2(spreedsheet))

Part One:  34581
Part Two:  214


<center><h1><a href='https://adventofcode.com/2017/day/3'>Day 3: Spiral Memory</a></h1></center>

In [9]:
UP    = np.array([ 0,-1])
DOWN  = np.array([ 0, 1])
LEFT  = np.array([-1, 0])
RIGHT = np.array([ 1, 0])
UP_RIGHT   = np.array([ 1,-1])
UP_LEFT    = np.array([-1,-1])
DOWN_RIGHT = np.array([ 1, 1])
DOWN_LEFT  = np.array([-1, 1])

def neighbours(v, diagonal = True):
    moves = [UP, DOWN, LEFT, RIGHT]
    if diagonal:
        moves += [UP_RIGHT, UP_LEFT, DOWN_RIGHT, DOWN_LEFT]
    return [v + m for m in moves]

### Part One:

In [10]:
def day_3_1(square_no):
    size = math.floor(math.sqrt(square_no)) + 2
    return abs((square_no - (size-2)**2) % (size-1) - size//2) + size//2

### Part Two:

In [11]:
def day_3_2(square_no):
    
    def coords():
        v = np.zeros(2, dtype=int)
        s = 0
        yield v
        while(True):
            for dv in [RIGHT, UP, LEFT, DOWN]:
                if dv[1]: s += 1
                for _ in range(s):
                    v += dv
                    yield v
    
    def values():
        fields = defaultdict(int)
        for v in coords():
            fields[tuple(v)] = sum([fields[tuple(u)] for u in neighbours(v)]) or 1
            yield fields[tuple(v)]
            
        
    
    return next(x for x in values() if x > square_no)

### Results:

In [12]:
square_no = int(get_input(3).readline().split()[0])

print('Part One: ', day_3_1(square_no))
print('Part Two: ', day_3_2(square_no))

Part One:  419
Part Two:  295229


<center><h1><a href='https://adventofcode.com/2017/day/4'>Day 4: High-Entropy Passphrases</a></h1></center>

### Part One:

In [13]:
def day_4_1(passphrases):
    return sum([len(p) == len(set(p)) for p in passphrases])

### Part Two:

In [14]:
def day_4_2(passphrases):    
    return sum([not any(sorted(a) == sorted(b) for (a,b) in combinations(phrase, 2)) for phrase in passphrases])

### Results:

In [15]:
passphrases = list(map(methodcaller("split"), get_input(4).readlines()))

print('Part One: ', day_4_1(passphrases))
print('Part Two: ', day_4_2(passphrases))

Part One:  477
Part Two:  167


<center><h1><a href='https://adventofcode.com/2017/day/5'>Day 5: A Maze of Twisty Trampolines, All Alike</a></h1></center>

### Part One:

In [16]:
def day_5_1(offsets):
    offsets = list(offsets)
    s = i = 0
    while(0 <= i < len(offsets)):
        offsets[i] += 1
        i += offsets[i] - 1
        s += 1
    return s;

### Part Two:

In [17]:
def day_5_2(offsets):
    offsets = list(offsets)
    s = i = 0
    while(0 <= i < len(offsets)):
        offset = offsets[i]
        offsets[i] += (-1 if offsets[i]>2 else 1)
        i += offset
        s += 1
    return s;

### Results:

In [18]:
offsets = list(map(int, get_input(5).readlines()))
print('Part One: ', day_5_1(offsets))
print('Part Two: ', day_5_2(offsets))

Part One:  359348
Part Two:  27688760


<center><h1><a href='https://adventofcode.com/2017/day/6'>Day 6: Memory Reallocation</a></h1></center>

### Part One:

In [19]:
def day_6_1(banks):
    
    def redistributions(banks):
        banks = list(banks)
        while True:
            i = np.argmax(banks)
            blocks = banks[i]
            banks[i] = 0
            while(blocks > 0):
                i += 1
                banks[(i)%len(banks)] += 1
                blocks -= 1
            yield tuple(banks)
    
    history = set(tuple(banks))
    cycle = 0
    for b in redistributions(banks):
        cycle += 1
        if b in history:
            return cycle
        history.add(b)

### Part Two:

In [20]:
def day_6_2(banks):
    
    def redistributions(banks):
        banks = list(banks)
        while True:
            i = np.argmax(banks)
            blocks = banks[i]
            banks[i] = 0
            while(blocks > 0):
                i += 1
                banks[(i)%len(banks)] += 1
                blocks -= 1
            yield tuple(banks)
    
    cycle = 0
    history = {tuple(banks): cycle}
    for b in redistributions(banks):
        cycle += 1
        if b in history:
            return cycle - history[b]
        history[b] = cycle

### Results:

In [21]:
banks = list(map(int, get_input(6).readline().split()))

print('Part One: ', day_6_1(banks))
print('Part Two: ', day_6_2(banks))

Part One:  3156
Part Two:  1610


<center><h1><a href='https://adventofcode.com/2017/day/7'>Day 7: Recursive Circus</a></h1></center>

In [22]:
class Program:
    
    def __init__(self, name, weight, parent = None, children = []):
        self.name = name
        self.weight = weight
        self.parent = parent
        self.children = children
    
    def tower_weight(self):
        return self.weight + sum(p.tower_weight() for p in self.children)
    
    @staticmethod
    def parse(line):
        line = line.replace(',', '').split()
        return Program(line[0], int(line[1][1:-1]), None, line[3:])
    
    @staticmethod
    def link_programs(programs):
        programs = {p.name: p for p in programs}
        for program in programs.values():
            for i in range(len(program.children)):
                program.children[i] = programs[program.children[i]]
                program.children[i].parent = program
        return next(p for p in programs.values() if p.parent == None)

### Part One:

In [23]:
def day_7_1(root):
    return root.name

### Part Two:

In [24]:
def day_7_2(root):
    
    def argdistinct(data):
        for i in range(len(data)):
            if data[i] != data[(i+1)%len(data)] and data[(i+1)%len(data)] == data[(i+2)%len(data)]:
                return i
        return None
    
    def balance(program):
        weights = [p.tower_weight() for p in program.children]
        i = argdistinct(weights)
        if i != None:
            b = balance(program.children[i])
            if b == 0:
                return program.children[i].weight - (weights[i] - weights[(i+1)%len(weights)])
            return b
        return 0
        
    return balance(root)

### Results:

In [25]:
root = Program.link_programs([Program.parse(l) for l in get_input(7).readlines()])

print('Part One: ', day_7_1(root))
print('Part Two: ', day_7_2(root))

Part One:  xegshds
Part Two:  299


<center><h1><a href='https://adventofcode.com/2017/day/8'>Day 8: I Heard You Like Registers</a></h1></center>

In [26]:
OPERATORS = {
    '>':  operator.gt,
    '<':  operator.lt,
    '>=': operator.ge,
    '<=': operator.le,
    '==': operator.eq,
    '!=': operator.ne
}

In [27]:
def Condition(register, operator, value):
    return lambda registers: operator(registers[register], value)

class Instruction(namedtuple('Instruction', ['register', 'change', 'condition'])):
    
    def execute(self, registers):
        if self.condition(registers):
            registers[self.register] += self.change
    
    @staticmethod
    def parse(line):
        line = line.split()
        return Instruction(
            line[0],
            int(line[2]) if line[1] == 'inc' else -int(line[2]),
            Condition(line[4], OPERATORS[line[5]], int(line[6]))
        )

### Part One:

In [28]:
def day_8_1(instructions):
    registers = defaultdict(int)
    for ins in instructions:
        ins.execute(registers)
    return max(registers.values())

### Part Two:

In [29]:
def day_8_2(instructions):
    registers = defaultdict(int)
    ans = 0
    for ins in instructions:
        ins.execute(registers)
        ans = max(ans, *registers.values())
    return ans

### Results:

In [30]:
instructions = list(Instruction.parse(line) for line in get_input(8).readlines())

print('Part One: ', day_8_1(instructions))
print('Part Two: ', day_8_2(instructions))

Part One:  3089
Part Two:  5391


<center><h1><a href='https://adventofcode.com/2017/day/9'>Day 9: Stream Processing</a></h1></center>

### Part One:

In [31]:
def day_9_1(file):
    score = 0
    indent = 1
    garbage = False
    ignore = False
    
    file.read(1)
    
    while True:
        c = file.read(1)
        
        if ignore:
            ignore = False
            continue
            
        if c == '!':
            ignore = True
            continue
        
        if garbage:
            if c == '>':
                garbage = False
            continue

        if c == '{':
            indent += 1
        elif c == '}':
            score += indent
            indent -= 1
        elif c == '<':
            garbage = True

        if indent == 0:
            return score

### Part Two:

In [32]:
def day_9_2(file):
    score = 0
    indent = 1
    garbage = False
    ignore = False
    
    file.read(1)
    
    while True:
        c = file.read(1)
        
        if ignore:
            ignore = False
            continue
            
        if c == '!':
            ignore = True
            continue
        
        if garbage:
            if c == '>':
                garbage = False
            else:
                score += 1
            continue

        if c == '{':
            indent += 1
        elif c == '}':
            indent -= 1
        elif c == '<':
            garbage = True

        if indent == 0:
            return score

### Results:

In [33]:
print('Part One: ', day_9_1(get_input(9)))
print('Part Two: ', day_9_2(get_input(9)))

Part One:  12803
Part Two:  6425


<center><h1><a href='https://adventofcode.com/2017/day/10'>Day 10: Knot Hash</a></h1></center>

In [34]:
def circural_reverse(data, start, length):
    data = data[start:] + data[:start]
    data[:length] = reversed(data[:length])
    data = data[-start:] + data[:-start]
    return data

def generate_spare_hash(lengths, rounds = 1, size = 256):
    data = list(range(size))
    index = 0
    skip = 0
    for _ in range(rounds):
        for l in lengths:
            data = circural_reverse(data, index, l)
            index = (index + l + skip) % size
            skip += 1
    return data

def generate_dense_hash(spare_hash, block = 16):
    return [reduce(operator.xor, spare_hash[i:i+block], 0) for i in range(0, len(spare_hash), block)]

def hash_as_hex(hash):
    return "".join(['{:02x}'.format(n) for n in hash])

def parse_lenghts(text):
    return [ord(c) for c in text] + [17, 31, 73, 47, 23]

def knot_hash(data, block = 16, rounds = 64, size = 256):
    return hash_as_hex(generate_dense_hash(generate_spare_hash(parse_lenghts(data), rounds, size), block))

In [35]:
assert circural_reverse(list(range(4)), 0, 3)  == [2, 1, 0, 3]
assert circural_reverse(list(range(5)), 0, 3)  == [2, 1, 0, 3, 4]
assert circural_reverse(list(range(5)), 4, 3)  == [0, 4, 2, 3, 1]
assert circural_reverse(list(range(5)), 3, 4)  == [4, 3, 2, 1, 0]

assert parse_lenghts('1,2,3') == [49,44,50,44,51,17,31,73,47,23]

assert generate_spare_hash([3, 4, 1, 5], rounds = 1, size = 5) == [3, 4, 2, 1, 0]

assert generate_dense_hash([65, 27, 9, 1, 4, 3, 40, 50, 91,  7, 6, 0, 2, 5, 68, 22]) == [64]

assert hash_as_hex([64, 7, 255]) == "4007ff"
assert hash_as_hex([255, 0, 17]) == 'ff0011'

assert knot_hash('') == 'a2582a3a0e66e6e86e3812dcb672a272'
assert knot_hash('63,144,180,149,1,255,167,84,125,65,188,0,2,254,229,24') == 'c500ffe015c83b60fad2e4b7d59dabc4'
assert knot_hash('AoC 2017') == '33efeb34ea91902bb2f59c9920caa6cd'
assert knot_hash('1,2,3') == '3efbe78a8d82f29979031a4aa0b16a9d'
assert knot_hash('1,2,4') == '63960835bcdc130f0b66d7ff4f6a5a8e'

### Part One:

In [36]:
def day_10_1(lengths):
    data = generate_spare_hash(lengths)
    return data[0] * data[1]

### Part Two:

In [37]:
def day_10_2(text):
    return knot_hash(text)

### Results:

In [38]:
lengths = list(map(int, get_input(10).readline().split(',')))

print('Part One: ', day_10_1(lengths))
print('Part Two: ', day_10_2(get_input(10).readline().strip()))

Part One:  1980
Part Two:  899124dac21012ebc32e2f4d11eaec55


<center><h1><a href='https://adventofcode.com/2017/day/11'>Day 11: Hex Ed</a></h1></center>

In [39]:
HEX_MOVES = {
    'n':  np.array([ 0,-1,-1]),
    'ne': np.array([ 1, 0,-1]),
    'se': np.array([ 1, 1, 0]),
    's':  np.array([ 0, 1, 1]),
    'sw': np.array([-1, 0, 1]),
    'nw': np.array([-1,-1, 0])
}       

### Part One:

In [40]:
def day_11_1(moves):
    return np.amax(np.abs(np.sum(moves, axis=0)))

### Part Two:

In [41]:
def day_11_2(moves):
    pos = np.array([0,0,0])
    dist = 0
    for m in moves:
        pos += m
        dist = max(dist, np.amax(np.abs(pos)))
    return dist

### Results:

In [42]:
moves = [HEX_MOVES[m] for m in get_input(11).readline().strip().split(",")]
#moves = [HEX_MOVES[m] for m in ['se','sw','se','sw','sw']]

print('Part One: ', day_11_1(moves))
print('Part Two: ', day_11_2(moves))

Part One:  685
Part Two:  1457


<center><h1><a href='https://adventofcode.com/2017/day/12'>Day 12: Digital Plumber</a></h1></center>

In [43]:
class Node:
    
    def __init__(self, id):
        self.id = id
        self.connections = []
        self.visited = False
    
    def dfs(self):
        if self.visited:
            return 0
        
        self.visited = True
        
        return 1 + sum([n.dfs() for n in self.connections])
    
    def __str__(self):
        return "{} {}".format(self.id, str(self.connections))
    
    @staticmethod
    def parse(line):
        line = line.replace(',', '').split()
        n = Node(int(line[0]))
        for i in range(2, len(line)):
            n.connections.append(int(line[i]))
        return n
    
    @staticmethod
    def connect_nodes(nodes):
        for n in nodes:
            for i in range(len(n.connections)):
                n.connections[i] = nodes[n.connections[i]]
        return nodes
    
    @staticmethod
    def set_visited(nodes, visited):
        for n in nodes:
            n.visited = visited

### Part One:

In [44]:
def day_12_1(nodes):
    Node.set_visited(nodes, False)
    return nodes[0].dfs()

### Part Two:

In [45]:
def day_12_2(nodes):
    Node.set_visited(nodes, False)
    return len([n.dfs() for n in nodes if not n.visited])

### Results:

In [46]:
nodes = Node.connect_nodes([Node.parse(line) for line in get_input(12).readlines()])

print('Part One: ', day_12_1(nodes))
print('Part Two: ', day_12_2(nodes))

Part One:  113
Part Two:  202


<center><h1><a href='https://adventofcode.com/2017/day/13'>Day 13: Packet Scanners</a></h1></center>

In [47]:
def scanner_position(size, time):
    if (size < 2):
        return 0
    if (time // (size - 1)) % 2 == 0:
        return time % (size - 1)
    else:
        return size - (time % (size - 1)) - 1
    
def severity(layers, delay = 0):
    ans = 0
    for i in range(0, max(layers)+1):
        if layers[i] > 0 and scanner_position(layers[i], delay + i) == 0:
            ans += i * layers[i]
    return ans

### Part One:

In [48]:
def day_13_1(layers):
    return severity(layers)

### Part Two:

In [49]:
def day_13_2(layers):
    for delay in count():
        if severity(layers, delay) == 0 and scanner_position(layers[0], delay) != 0:
            return delay

### Results:

In [50]:
layers = defaultdict(int, {int(l[0]): int(l[1]) for l in map(lambda line: line.replace(':','').split(), get_input(13).readlines())})

print('Part One: ', day_13_1(layers))
print('Part Two: ', day_13_2(layers))

Part One:  1728
Part Two:  3946838


<center><h1><a href='https://adventofcode.com/2017/day/14'>Day 14: Disk Defragmentation</a></h1></center>

In [51]:
def hex_to_binary(text):
    return "".join([bin(int(c, 16))[2:].zfill(4) for c in text])

def count_chars(text, char):
    return len([c for c in text if c == char])

def is_in_range(v, a, b):
    return a <= v and v < b

def clear_memory(memory, v):
    if not is_in_range(v[0], 0, 128) or not is_in_range(v[1], 0, 128) or memory[v[0]][v[1]] == '0':
        return
    memory[v[0]][v[1]] = '0'
    for n in neighbours(v, False):
        clear_memory(memory, n)
        

In [52]:
assert hex_to_binary('f') == '1111'
assert hex_to_binary('1a') == '00011010'
assert hex_to_binary('00') == '00000000'
assert hex_to_binary('a0c2017') == '1010000011000010000000010111'

assert count_chars('11010', '1') == 3

### Part One:

In [53]:
def day_14_1(key):
    return sum(count_chars(hex_to_binary(knot_hash(key+'-'+str(i))), '1') for i in range(128))

### Part Two:

In [54]:
def day_14_2(key):
    memory = [list(hex_to_binary(knot_hash(key+'-'+str(i)))) for i in range(128)]
    ans = 0
    for i in range(128):
        for j in range(128):
            if memory[i][j] == '1':
                clear_memory(memory, np.array([i,j]))
                ans += 1
    return ans

### Results:

In [55]:
key = get_input(14).readline().strip()

print('Part One: ', day_14_1(key))
print('Part Two: ', day_14_2(key))

Part One:  8226
Part Two:  1128


<center><h1><a href='https://adventofcode.com/2017/day/15'>Day 15: Dueling Generators</a></h1></center>

In [56]:
def generator(previous, factor, divider = 2147483647):
    while True:
        previous = (previous * factor) % divider
        yield previous

def generatorA(start): return generator(start, 16807)
def generatorB(start): return generator(start, 48271)

def judge(generator1, generator2, tests, bits = 16):
    d = 1 << bits
    return sum(a%d == b%d for (_, a, b) in zip(range(tests), generator1, generator2))

def criteria(generator, value):
    return (n for n in generator if n%value == 0)

In [57]:
assert list(islice(generatorA(65), 5)) == [1092455, 1181022009, 245556042, 1744312007, 1352636452]
assert list(islice(generatorB(8921), 5)) == [430625591, 1233683848, 1431495498, 137874439, 285222916]

assert judge(generatorA(65), generatorB(8921), 5) == 1
assert judge(generatorA(65), generatorB(8921), 40*10**6) == 588

assert list(islice(criteria(generatorA(65), 4), 5)) == [1352636452, 1992081072, 530830436, 1980017072, 740335192]
assert list(islice(criteria(generatorB(8921), 8), 5)) == [1233683848, 862516352, 1159784568, 1616057672, 412269392]

assert judge(criteria(generatorA(65), 4), criteria(generatorB(8921), 8), 1055) == 0
assert judge(criteria(generatorA(65), 4), criteria(generatorB(8921), 8), 1056) == 1
assert judge(criteria(generatorA(65), 4), criteria(generatorB(8921), 8), 5*10**6) == 309

### Part One:

In [58]:
def day_15_1(startA, startB):
    return judge(generatorA(startA), generatorB(startB), 40*10**6)

### Part Two:

In [59]:
def day_15_2(startA, startB):
    return judge(criteria(generatorA(startA), 4), criteria(generatorB(startB), 8), 5*10**6)

### Results:

In [60]:
[startA, startB] = [int(l.split()[4]) for l in get_input(15).readlines()]

print('Part One: ', day_15_1(startA, startB))
print('Part Two: ', day_15_2(startA, startB))

Part One:  600
Part Two:  313


---

<center><h1><a href='https://adventofcode.com/2017/day/'></a></h1></center>

### Part One:

In [61]:
def day__1():
    pass

### Part Two:

In [62]:
def day__2():
    pass

### Results:

In [63]:

print('Part One: ', day__1())
print('Part Two: ', day__2())

Part One:  None
Part Two:  None
